In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [3]:
now = datetime.now()
format_ES = "%Y.%m.%d"
fecha_hoy = str(now.strftime(format_ES))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))

### Definiendo indice principal con fecha de hoy

In [4]:
indice = parametros.mintic_concat_index + '-' + fecha_hoy

### Función para generar JSON compatible con ES

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [6]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_gestion_incidentes"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['gestion.fechaControl']
except:
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-05-01T00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [88]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    exit()

In [90]:
{col:datos_semilla[datos_semilla[col]==''].shape[0] for col in datos_semilla.columns}

{'nombreSede': 0,
 'longitud': 270,
 'latitud': 270,
 'COD_ISO': 0,
 'codDanesede': 0,
 'nombre_centro_pob': 0,
 'site_id': 5129,
 'nombre_departamento': 0,
 'energiadesc': 0,
 'nombre_municipio': 2,
 'id_Beneficiario': 0}

In [ ]:
datos_semilla['longitud']

In [54]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.location']=datos_semilla['gestion.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In [80]:
datos_semilla.head(20)

,nombreSede,COD_ISO,codDanesede,nombre_centro_pob,site_id,nombre_departamento,energiadesc,nombre_municipio,id_Beneficiario,gestion.location
0,C.E.R LA PAULINA,CO-ANT,205854000501,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469"
1,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,205284000201,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667"
2,C. E. R. MERCEDES YEPES,CO-ANT,205040000161,VDA. LA MESETA,,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18787,"6.93848274,-75.151177190"
3,C. E. R. MEDIAS FALDAS,CO-ANT,205040000616,VDA EL LIMÓN,,ANTIOQUIA,TIENE ENERGÍA,ANORÍ,18802,"7.04797951,-75.19442611"
4,C. E. R. ROSAURA ORTEGA,CO-ANT,205040000641,BELLAVISTA,,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18804,"7.06272584,-75.22730972"
5,C. E. R. CHAGUALO ABAJO,CO-ANT,205040001035,VDA CHAGUALO ABAJO,,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18820,"6.228333,-76.006111"
6,C. E. R. EL CUCHUCO,CO-ANT,205093000308,VDA CUCHUCO,,ANTIOQUIA,TIENE ENERGÍA,BETULIA,19170,"6.10852897,-75.9849316"
7,C. E. R. LA CIBELES,CO-ANT,205093000324,VDA LA CIBELES,,ANTIOQUIA,RED INTERCONECTADA,BETULIA,19172,"6.14615356,-75.93353968"
8,I. E. R. LA SUCRE,CO-ANT,205093000413,VDA LA SUCRE,,ANTIOQUIA,TIENE ENERGÍA,BETULIA,19176,"6.1175,-75.93583333"
9,C. E. R. SANTA RITA,CO-ANT,205093000529,VDA. SANTA RITA,,ANTIOQUIA,RED INTERCONECTADA,BETULIA,19179,"6.23676381,-76.03442515"


### Leyendo indice servicemanager-incidentes

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* clr_bmc_location, es la llave para cruzar con cada centro de conexión.
* product_type, es el detalle del ticket.
* clr_bmc_host, estado la IP
* assignment, es el sujeto asignado al ticket
* contact_name, usuario del ticket
* open_time y resolved_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* number, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* category y subcategory, clasificacion dada por mintic

In [86]:
def traeSMIncidentes(fecha_max,fecha_tope):
    print(fecha_max,fecha_tope)
    total_docs = 100
    try:
        response = es.search(
            index= parametros.sm_incidentes,
            body={
                "_source": ["clr_bmc_location","product_type","clr_bmc_host","resolution","contact_name"
                            ,"assignment","open_time","resolved_time","resolution","number","severity"]
                ,"query": {
                    "range": {
                      "open_time": {
                        "gte": fecha_max,
                        "lt": fecha_tope
                      }
                    }
                }
            },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]
        fields = {}
        for num, doc in enumerate(elastic_docs):
            source_data = doc["_source"]
            for key, val in source_data.items():
                try:
                    fields[key] = np.append(fields[key], val)
                except KeyError:
                    fields[key] = np.array([val])

        datos_SM_incidents = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
        print(datos_SM_incidents)
        datos_SM_incidents['gestion.categoria'] = 'Dano' 
        datos_SM_incidents['gestion.subcategoria'] = 'Dano masivo internet' 
        datos_SM_incidents = datos_SM_incidents.rename(columns={'severity':'gestion.gravedad','product_type' : 'gestion.detallesTicket','contact_name':'gestion.usuarioTicket','assignment':'gestion.responsable','clr_bmc_location':'site_id','clr_bmc_host':'gestion.IP','number':'gestion.numeroTicket'})
        #datos_SM_incidents['gestion.tiempoRespuesta'] = datos_SM_incidents.apply(lambda row: row.resolved_time - row.open_time, axis=1)
        datos_SM_incidents['gestion.gravedad'] = datos_SM_incidents['gestion.gravedad'].replace(['1','2','3'],['Alto','Medio','Bajo'])
        return datos_SM_incidents
    except:
        return pd.DataFrame()

Realizando bucle hasta conseguir datos de servicemanager-incidentes o hasta la fecha actual para realizar la carga de datos

In [91]:
fecha_max_mintic = fecha_ejecucion
fecha_max_mintic = "2021-04-23T19:10:00"
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)

#if datos_SM_incidents is None or datos_SM_incidents.empty:
#    while (datos_SM_incidents is None or datos_SM_incidents.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
#        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
#        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
#        datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)
#else:
#    pass

2021-04-23T19:10:00 2021-04-23T19:19:59
  severity     number      contact_name      product_type       assignment  \
0        1  IM1278675  bmcintegrationsm  hardware failure  EYN - NOCMINTIC   

     clr_bmc_host                 open_time clr_bmc_location resolved_time  \
0  172.28.107.185  2021-04-23T19:15:15.000Z    31699-ZGYO293          None   

  resolution  
0       None  


Haciendo merge entre semilla e incidentes

In [92]:
try:
    concat = pd.merge(datos_SM_incidents,datos_semilla, on=['site_id'],how='inner')
    concat = concat.rename(columns={'id_Beneficiario' : 'gestion.id_Beneficiario','resolution':'gestion.resolucion','open':'gestion.abierto','open_time':'gestion.fechaApertura','resolved_time':'gestion.fechaCierre','nombreSede':'gestion.nombreSede','site_id':'gestion.site_id','nombre_departamento':'gestion.dptoGestion','nombre_municipio':'gestion.muniGestion','nombre_centro_pob':'gestion.nombre_centro_pob','COD_ISO':'gestion.COD_ISO','codDanesede':'gestion.codDanesede','energiadesc':'gestion.energiadesc'})
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.replace("T"," ")
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.slice(stop=19)
    concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.replace("T"," ")
    concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.slice(stop=19)
    concat["gestion.anyo"] = concat["gestion.fechaApertura"].str[0:4]
    concat["gestion.mes"] = concat["gestion.fechaApertura"].str[5:7]
    concat["gestion.dia"] = concat["gestion.fechaApertura"].str[8:10]
    concat["gestion.hora"] = concat["gestion.fechaApertura"].str[11:13]
    concat["gestion.min"] = concat["gestion.fechaApertura"].str[14:16]
    concat["gestion.seg"] = concat["gestion.fechaApertura"].str[17:19]
except:
    concat = pd.DataFrame()

Realizando inserción

In [93]:
use_these_keys = ['gestion.numeroTicket'
                  , 'gestion.detallesTicket'
                  , 'gestion.usuarioTicket'
                  , 'gestion.categoria'
                  , 'gestion.subcategoria'
                  , 'gestion.fechaApertura'
                  , 'gestion.fechaCierre'
                  , 'gestion.responsable'
                  , 'gestion.id_Beneficiario'
                  , 'gestion.resolucion'
                  , 'gestion.nombreSede'
                  , 'gestion.COD_ISO'
                  , 'gestion.site_id'
                  , 'gestion.dptoGestion'
                  , 'gestion.muniGestion'
                  , 'gestion.energiadesc'
                  , 'gestion.gravedad'
                  , 'gestion.location'
                  , 'gestion.IP'
                  , 'gestion.anyo'
                  , 'gestion.mes'
                  , 'gestion.dia'
                  , 'gestion.hora'
                  , 'gestion.min'
                  , 'gestion.seg'
                  , '@timestamp']

concat['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{str(document['gestion.site_id']) + '-' + str(document['gestion.numeroTicket']) + '-' + str(document['gestion.fechaApertura'])}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(concat))
print("Fecha: ", now,"- Valoraciones insertadas en indice principal:",salida[0])

Fecha:  2021-06-03 20:50:38.709167 - Valoraciones insertadas en indice principal: 1


Actualizando fecha de control de ejecución

In [95]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = "2021-04-20T19:10:00"

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia_gestion_incidentes',
        body = { 'jerarquia_gestion_incidentes': 'gestion_incindentes','gestion.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-04-23T19:20:00
